In [8]:
import pandas as pd
import numpy as np 
from tqdm.notebook import tqdm
import os, sys 

In [3]:
os.getcwd()

'c:\\Users\\EonKim\\Desktop\\github\\Recommend System\\10k'

In [23]:
books = pd.read_csv('dataset\\books.csv')
book_tags = pd.read_csv('dataset\\book_tags.csv')
ratings = pd.read_csv('dataset\\ratings.csv')
test = pd.read_csv('dataset/test.csv')
train = pd.read_csv('dataset/train.csv')
tags = pd.read_csv('dataset\\tags.csv')
to_read = pd.read_csv('dataset\\to_read.csv')

In [24]:
rec_df = pd.DataFrame()
rec_df['user_id'] = train['user_id'].unique()

In [25]:
sol = test.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
gt = {}
for user in tqdm(sol['user_id'].unique()): 
    gt[user] = list(sol[sol['user_id'] == user]['unique'].values[0])

In [26]:
popular_rec_model = books.sort_values(by = 'books_count', ascending = False)['book_id'].values[:500]

In [28]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    for rec in popular_rec_model[0:200]:
        rec_list.append(rec)
    total_rec_list[user] = rec_list 

In [31]:
import six
import math

# https://github.com/kakao-arena/brunch-article-recommendation/blob/master/evaluate.py

class evaluate():
    def __init__(self, recs, gt, topn=100):
        self.recs = recs
        self.gt = gt 
        self.topn = topn 
        
    def _ndcg(self):
        Q, S = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            dcg = 0.0
            idcg = sum([1.0 / math.log(i + 2, 2) for i in range(min(len(seen), len(rec)))])
            for i, r in enumerate(rec):
                if r not in seen:
                    continue
                rank = i + 1
                dcg += 1.0 / math.log(rank + 1, 2)
            ndcg = dcg / idcg
            S += ndcg
            Q += 1
        return S / Q


    def _map(self):
        n, ap = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            _ap, correct = 0.0, 0.0
            for i, r in enumerate(rec):
                if r in seen:
                    correct += 1
                    _ap += (correct / (i + 1.0))
            _ap /= min(len(seen), len(rec))
            ap += _ap
            n += 1.0
        return ap / n


    def _entropy_diversity(self):
        sz = float(len(self.recs)) * self.topn
        freq = {}
        for u, rec in six.iteritems(self.recs):
            for r in rec:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent
    
    def _evaluate(self):
        print('MAP@%s: %s' % (self.topn, self._map()))
        print('NDCG@%s: %s' % (self.topn, self._ndcg()))
        print('EntDiv@%s: %s' % (self.topn, self._entropy_diversity()))

In [32]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.0
NDCG@200: 0.0
EntDiv@200: 5.298317366548021


In [33]:
train = pd.merge(train, books[['book_id', 'authors', 'ratings_count']], how='left', on='book_id')

In [34]:
agg = train.groupby(['user_id','authors'])['authors'].agg({'count'}).reset_index()
agg = agg.sort_values(by='count', ascending=False)
agg.head()

,user_id,authors,count
5866,9731,Bill Bryson,8
19610,32055,Bill Bryson,8
17669,28600,Michael Crichton,7
590,1185,Arthur Conan Doyle,6
6282,10335,"Johanna Spyri, Angelo Rinaldi, Beverly Cleary",6


In [35]:
author_books = books[['book_id', 'authors', 'ratings_count']].sort_values(by=['authors', 'ratings_count'], ascending=[True, False])
author_books = author_books.reset_index(drop=True)

author_books.head()

,book_id,authors,ratings_count
0,101911,A. Manette Ansay,23966
1,16081754,A. Meredith Walters,24615
2,16102880,A. Meredith Walters,24287
3,99107,"A.A. Milne, Ernest H. Shepard",207550
4,776407,"A.A. Milne, Ernest H. Shepard",69102


In [36]:
author_rec_model = agg.merge(author_books, how='left', on=['authors'])
author_rec_model.head()

,user_id,authors,count,book_id,ratings_count
0,9731,Bill Bryson,8,9791,250192
1,9731,Bill Bryson,8,21,191535
2,9731,Bill Bryson,8,28,66947
3,9731,Bill Bryson,8,24,58688
4,9731,Bill Bryson,8,7507825,55296


In [37]:
author_rec_model[author_rec_model['user_id'] == 30944]['book_id'].values

array([    9791,       21,       28,       24,  7507825,       25,
          10538,       27,       26, 17262366,   135611,       29,
       25489012,    10541,     1715,      446,    50618,      447,
        6345193,      432,      456,    10006,      449,        5,
            599,      597,    24475,  6386555,    24476,  2159007,
           1633], dtype=int64)

In [38]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    for rec in author_rec_model_: 
        rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            rec_list.append(rec)
        
    total_rec_list[user] = rec_list[0:200]

In [39]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.08722925928891606
NDCG@200: 0.018830978525207494
EntDiv@200: 5.8416214483807245


In [40]:
read_list = train.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
read_list.head()

,user_id,unique
0,1,[4893]
1,2,[8034]
2,3,[9014]
3,4,[8464]
4,5,"[6703, 4829]"


In [41]:
total_rec_list = {}
for user in tqdm(rec_df['user_id'].unique()):
    rec_list = []
    author_rec_model_ = author_rec_model[author_rec_model['user_id'] == user]['book_id'].values
    seen = read_list[read_list['user_id'] == user]['unique'].values[0]
    for rec in author_rec_model_: 
        if rec not in seen:
            rec_list.append(rec)
    
    if len(rec_list) < 200:
        for i in popular_rec_model[0:200]:
            if rec not in seen:
                rec_list.append(rec)

    total_rec_list[user] = rec_list[0:200]

In [42]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 0.05637773079633545
NDCG@200: 0.012035855451876305
EntDiv@200: 5.076768062387068


In [43]:
language = pd.merge(train, books[['book_id', 'language_code']], how='left', on='book_id')

In [44]:
language_list = language.groupby(['user_id'])['language_code'].agg({'unique'}).reset_index()
language_list.head()

,user_id,unique
0,1,[nan]
1,2,[nan]
2,3,[eng]
3,4,[nan]
4,5,[nan]
